# Example notebook using Qililab

In [1]:
import os
from pathlib import Path

from qililab import Experiment, build_platform

[Qibo 0.1.7|INFO|2022-11-27 17:17:21]: Using numpy backend on /CPU:0
[Qibo 0.1.7|WARNING|2022-11-27 17:17:21]: numpy backend uses `np.einsum` and supports CPU only. Consider installing the qibojit or qibotf backends for increased performance and to enable GPU acceleration.


In [2]:
fname = os.path.abspath("")
os.environ["RUNCARDS"] = str(Path(fname) / "runcards")
os.environ["DATA"] = str(Path(fname) / "data")

## Load a platform

In [3]:
runcard_name = "template_base_runcard_soprano"
platform = build_platform(name=runcard_name)

[qililab] [0.9.2|INFO|2022-11-27 17:17:27]: Building platform
INFO:qililab.config.config:Building platform


In [4]:
print(platform.chip)

Chip soprano with 7 qubits and 18 ports: 

 * Port 1: ------
 * Port 2: ----|qubit_2|----
 * Port 3: ----|resonator_q6|--|resonator_q4|--|resonator_q3|--|resonator_isolated|--|resonator_q0|--|resonator_q2|--|resonator_q1|--|resonator_q5|----
 * Port 4: ------
 * Port 5: ----|qubit_4|----
 * Port 6: ----|qubit_4|----
 * Port 7: ----|qubit_2|----
 * Port 8: ----|qubit_1|----
 * Port 9: ----|qubit_1|----
 * Port 10: ------
 * Port 11: ----|resonator_q5|--|resonator_q1|--|resonator_q2|--|resonator_q0|--|resonator_isolated|--|resonator_q3|--|resonator_q4|--|resonator_q6|----
 * Port 12: ------
 * Port 13: ------
 * Port 14: ----|qubit_0|----
 * Port 15: ----|qubit_0|----
 * Port 16: ------
 * Port 17: ----|qubit_3|----
 * Port 18: ----|qubit_3|----



### Create an experiment

[qililab] [0.7.3|INFO|2022-10-04 11:52:15]: Building platform
INFO:qililab.config.config:Building platform


### Execute an experiment